In [ ]:
import numpy as np
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import csv
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import random
import time
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
length=1648
data_path='.../data2.xlsx'
save_path='...'
data_path_add='.../4d_drills.xlsx'
save_path_add='...'
distance_path='.../distance.csv'
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.size'] = 15
plt.rcParams['font.family'] = 'Arial'
font={'family' : 'Arial',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 15,
        }

In [ ]:
#creater data.npz
All_data=np.asarray(pd.read_excel(data_path,sheet_name='Sheet1'))
soil=All_data[0:length,4].astype('float') 
w=All_data[0:length,5].astype('float') 
rou=All_data[0:length,6].astype('float') 
e=All_data[0:length,7].astype('float') 
Sr=All_data[0:length,8].astype('float') 
cord=All_data[0:length,9].astype('float') 
Es=All_data[0:length,10].astype('float') 
out_data=np.zeros((length,1,6))
out_data[:,0,0]=Es
out_data[:,0,1]=soil
out_data[:,0,2]=w
out_data[:,0,3]=e
out_data[:,0,4]=Sr
out_data[:,0,5]=rou
data_path_npz=save_path+'/data.npz'
np.savez(data_path_npz,data=out_data)

out_data_2=np.zeros((length,1,6))
out_data_2[:,0,0]=Es
out_data_2[:,0,1]=soil
out_data_2[:,0,2]=w
out_data_2[:,0,3]=e
out_data_2[:,0,4]=Sr
out_data_2[:,0,5]=cord
data_path_npz_2=save_path+'/data2.npz'
#np.savez(data_path_npz_2,data=out_data_2)

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_c, hid_c, out_c):
        """
        GCN
        :param in_c: input channels
        :param hid_c:  hidden nodes
        :param out_c:  output channels
        """
        super(GCN, self).__init__()
        self.linear_1 = nn.Linear(in_c, hid_c)
        self.linear_2 = nn.Linear(hid_c, out_c)
        self.act = nn.ReLU()
    def forward(self, data):
        graph_data = data["graph"][0]  # [N, N]
        graph_data = self.process_graph(graph_data)
        flow_x = data["flow_x"]  # [B, N, H, D]
        B, N = flow_x.size(0), flow_x.size(1)
        flow_x = flow_x.view(B, N, -1)  # [B, N, H*D]  H = 6, D = 1
        output_1 = self.linear_1(flow_x)  # [B, N, hid_C]
        output_1 = self.act(torch.matmul(graph_data, output_1))  # [N, N], [B, N, Hid_C]
        output_2 = self.linear_2(output_1)
        output_2 = self.act(torch.matmul(graph_data, output_2))  # [B, N, 1, Out_C]
        return output_2.unsqueeze(2)
    @staticmethod
    def process_graph(graph_data):
        N = graph_data.size(0)
        matrix_i = torch.eye(N, dtype=graph_data.dtype, device=graph_data.device)
        graph_data += matrix_i  # A~ [N, N]
        degree_matrix = torch.sum(graph_data, dim=-1, keepdim=False)  # [N]
        degree_matrix = degree_matrix.pow(-1)
        degree_matrix[degree_matrix == float("inf")] = 0.  # [N]
        degree_matrix = torch.diag(degree_matrix)  # [N, N]
        return torch.mm(degree_matrix, graph_data)  # D^(-1) * A = \hat(A)
class ChebConv(nn.Module):
    def __init__(self, in_c, out_c, K, bias=True, normalize=True):
        """
        ChebNet conv
        :param in_c: input channels
        :param out_c:  output channels
        :param K: the order of Chebyshev Polynomial
        :param bias:  if use bias
        :param normalize:  if use norm
        """
        super(ChebConv, self).__init__()
        self.normalize = normalize
        self.weight = nn.Parameter(torch.Tensor(K + 1, 1, in_c, out_c))  # [K+1, 1, in_c, out_c]
        init.xavier_normal_(self.weight)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(1, 1, out_c))
            init.zeros_(self.bias)
        else:
            self.register_parameter("bias", None)
        self.K = K + 1
    def forward(self, inputs, graph):
        """
        :param inputs: he input data, [B, N, C]
        :param graph: the graph structure, [N, N]
        :return: convolution result, [B, N, D]
        """
        L = ChebConv.get_laplacian(graph, self.normalize)  # [N, N]
        mul_L = self.cheb_polynomial(L).unsqueeze(1)  # [K, 1, N, N]
        result = torch.matmul(mul_L, inputs)  # [K, B, N, C]
        result = torch.matmul(result, self.weight)  # [K, B, N, D]
        result = torch.sum(result, dim=0) + self.bias  # [B, N, D]
        return result
    def cheb_polynomial(self, laplacian):
        """
        Compute the Chebyshev Polynomial, according to the graph laplacian

        :param laplacian: the multi order Chebyshev laplacian, [K, N, N]
        :return:
        """
        N = laplacian.size(0)  # [N, N]
        multi_order_laplacian = torch.zeros([self.K, N, N], device=laplacian.device, dtype=torch.float)  # [K, N, N]
        multi_order_laplacian[0] = torch.eye(N, device=laplacian.device, dtype=torch.float)
        if self.K == 1:
            return multi_order_laplacian
        else:
            multi_order_laplacian[1] = laplacian
            if self.K == 2:
                return multi_order_laplacian
            else:
                for k in range(2, self.K):
                    multi_order_laplacian[k] = 2 * torch.mm(laplacian, multi_order_laplacian[k - 1]) - \
                                               multi_order_laplacian[k - 2]
        return multi_order_laplacian
    @staticmethod
    def get_laplacian(graph, normalize):
        """
        compute the laplacian of the graph
        :param graph: the graph structure without self loop, [N, N]
        :param normalize: whether to used the normalized laplacian
        :return:
        """
        if normalize:
            D = torch.diag(torch.sum(graph, dim=-1) ** (-1 / 2))
            L = torch.eye(graph.size(0), device=graph.device, dtype=graph.dtype) - torch.mm(torch.mm(D, graph), D)
        else:
            D = torch.diag(torch.sum(graph, dim=-1))
            L = D - graph
        return L
class ChebNet(nn.Module):
    def __init__(self, in_c, hid_c, out_c, K):
        """
        :param in_c: int, number of input channels.
        :param hid_c: int, number of hidden channels.
        :param out_c: int, number of output channels.
        :param K:
        """
        super(ChebNet, self).__init__()
        self.conv1 = ChebConv(in_c=in_c, out_c=hid_c, K=K)
        self.conv2 = ChebConv(in_c=hid_c, out_c=out_c, K=K)
        self.act = nn.ReLU()
    def forward(self, data):
        graph_data = data["graph"][0]  # [N, N]
        flow_x = data["flow_x"]  # [B, N, H, D]
        B, N = flow_x.size(0), flow_x.size(1)
        flow_x = flow_x.view(B, N, -1)  # [B, N, H*D]
        output_1 = self.act(self.conv1(flow_x, graph_data))
        output_2 = self.act(self.conv2(output_1, graph_data))
        return output_2.unsqueeze(2)
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_c, out_c, alpha=0.2):
        """
        graph attention layer
        :param in_c:
        :param out_c:
        :param alpha:
        """
        super(GraphAttentionLayer, self).__init__()
        self.in_c = in_c
        self.out_c = out_c
        self.alpha = alpha
        self.W = nn.Parameter(torch.empty(size=(in_c, out_c)))
        nn.init.xavier_normal_(self.W.data)
        self.a = nn.Parameter(torch.empty(size=(2 * out_c, 1)))
        nn.init.xavier_normal_(self.a.data)
        self.leakyrelu = nn.LeakyReLU(self.alpha)
    def forward(self, features, adj):
        B, N = features.size(0), features.size(1)
        adj = adj + torch.eye(N, dtype=adj.dtype).cuda()  # A+I
        h = torch.matmul(features, self.W)  # [B,N,out_features]
        # [B, N, N, 2 * out_features]
        a_input = torch.cat([h.repeat(1, 1, N).view(B, N * N, -1), h.repeat(1, N, 1)], dim=2).view(B, N, -1, 2 * self.out_c)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(3))  # [B,N, N, 1] => [B, N, N]
        zero_vec = -1e12 * torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)  # [B,N,N]
        attention = F.softmax(attention, dim=2)  # softmax [N, N]
        h_prime = torch.matmul(attention, h)  # [B,N, N]*[N, out_features] => [B,N, out_features]
        return h_prime
    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'
class GAT(nn.Module):
    def __init__(self, in_c, hid_c, out_c, n_heads=6):
        """
        :param in_c: int, number of input channels.
        :param hid_c: int, number of hidden channels.
        :param out_c: int, number of output channels.
        :param n_heads: how many heads
        """
        super(GAT, self).__init__()
        self.attentions = nn.ModuleList([GraphAttentionLayer(in_c, hid_c) for _ in range(n_heads)])
        self.conv2 = GraphAttentionLayer(hid_c*n_heads, out_c)
        self.act = nn.ELU()
    def forward(self, data):
        # data prepare
        adj = data["graph"][0]  # [N, N]
        x = data["flow_x"]  # [B, N, H, D]
        B, N = x.size(0), x.size(1)
        x = x.view(B, N, -1)  # [B, N, H*D]
        # forward
        outputs = self.act(torch.cat([attention(x, adj) for attention in self.attentions], dim=-1))
        output_2 = self.act(self.conv2(outputs, adj))
        return output_2.unsqueeze(2)  # [B,1,N,1]
    
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        x=data["flow_x"]
        B,N=x.size(0), x.size(1)
        x = x.view(B, N, -1) 
        return self.linear(x)
        
def MAE(y_true, y_pre):
    y_true = (y_true).detach().numpy().copy().reshape((-1, 1))
    y_pre = (y_pre).detach().numpy().copy().reshape((-1, 1))
    re = np.abs(y_true - y_pre).mean()
    return re
def RMSE(y_true, y_pre):
    y_true = (y_true).detach().numpy().copy().reshape((-1, 1))
    y_pre = (y_pre).detach().numpy().copy().reshape((-1, 1))
    re = math.sqrt(((y_true - y_pre) ** 2).mean())
    return re
def MAPE(y_true, y_pre):
    y_true = (y_true).detach().numpy().copy().reshape((-1, 1))
    y_pre = (y_pre).detach().numpy().copy().reshape((-1, 1))
    e = (y_true + y_pre) / 2 + 1e-2
    re = (np.abs(y_true - y_pre) / (np.abs(y_true) + e)).mean()
    return re
def R2(y_true, y_pre):
    y_true = (y_true).detach().numpy().copy().reshape((-1, 1))
    y_pre = (y_pre).detach().numpy().copy().reshape((-1, 1))
    re = (np.abs(r2_score(y_true,y_pre))).mean()
    return re

In [ ]:
def get_adjacent_matrix(distance_file: str, num_nodes: int, id_file: str = None, graph_type="connect") -> np.array:
    """
    construct adjacent matrix by csv file
    :param distance_file: path of csv file to save the distances between nodes
    :param num_nodes: number of nodes in the graph
    :param id_file: path of txt file to save the order of the nodes
    :param graph_type: ["connect", "distance"] if use weight, please set distance
    :return:
    """
    A = np.zeros([int(num_nodes), int(num_nodes)])
    if id_file:
        with open(id_file, "r") as f_id:
            node_id_dict = {int(node_id): idx for idx, node_id in enumerate(f_id.read().strip().split("\n"))}

            with open(distance_file, "r") as f_d:
                f_d.readline()
                reader = csv.reader(f_d)
                for item in reader:
                    if len(item) != 3:
                        continue
                    i, j, distance = int(item[0]), int(item[1]), float(item[2])
                    if graph_type == "connect":
                        A[node_id_dict[i], node_id_dict[j]] = 1.
                        A[node_id_dict[j], node_id_dict[i]] = 1.
                    elif graph_type == "distance":
                        A[node_id_dict[i], node_id_dict[j]] = 1. / distance
                        A[node_id_dict[j], node_id_dict[i]] = 1. / distance
                    else:
                        raise ValueError("graph type is not correct (connect or distance)")
        return A
    with open(distance_file, "r") as f_d:
        f_d.readline()
        reader = csv.reader(f_d)
        for item in reader:
            if len(item) != 3:
                continue
            i, j, distance = int(item[0]), int(item[1]), float(item[2])

            if graph_type == "connect":
                A[i, j], A[j, i] = 1., 1.
            elif graph_type == "distance":
                A[i, j] = 1. / distance
                A[j, i] = 1. / distance
            else:
                raise ValueError("graph type is not correct (connect or distance)")
    return A
def get_flow_data(flow_file: str) -> np.array:
    """
    parse npz to get flow data
    :param flow_file: (N, T, D)
    :return:
    """
    data = np.load(flow_file)
    flow_data = data['data'].transpose([1, 0, 2])[:, :, 0][:, :, np.newaxis]  # [N, T, D]  D = 1
    return flow_data

In [ ]:
class PEMSDataset(Dataset):
    def __init__(self, data_path, num_nodes, divide_days, time_interval, history_length, train_mode):
        """
        load processed data
        :param data_path: ["graph file name" , "flow data file name"], path to save the data file names
        :param num_nodes: number of nodes in graph
        :param divide_days: [ days of train data, days of test data], list to divide the original data
        :param time_interval: time interval between two traffic data records (mins)
        :param history_length: length of history data to be used
        :param train_mode: ["train", "test"]
        """
        self.data_path = data_path
        self.num_nodes = num_nodes
        self.train_mode = train_mode
        self.train_days = divide_days[0]
        self.test_days = divide_days[1]
        self.history_length = history_length  # 6
        self.time_interval = time_interval  # 5 min
        self.one_day_length = int(80/ self.time_interval)
        self.graph = get_adjacent_matrix(distance_file=data_path[0], num_nodes=num_nodes)
        self.flow_norm, self.flow_data = self.pre_process_data(data=get_flow_data(data_path[1]), norm_dim=1)
    def __len__(self):
        if self.train_mode == "train":
            return self.train_days * self.one_day_length - self.history_length
        elif self.train_mode == "test":
            return self.test_days * self.one_day_length
        else:
            raise ValueError("train mode: [{}] is not defined".format(self.train_mode))
    def __getitem__(self, index):  # (x, y), index = [0, L1 - 1]
        if self.train_mode == "train":
            index = index
        elif self.train_mode == "test":
            index += self.train_days * self.one_day_length
        else:
            raise ValueError("train mode: [{}] is not defined".format(self.train_mode))

        data_x, data_y = PEMSDataset.slice_data(self.flow_data, self.history_length, index, self.train_mode)
        data_x = PEMSDataset.to_tensor(data_x)  # [N, H, D]
        data_y = PEMSDataset.to_tensor(data_y).unsqueeze(1)  # [N, 1, D]
        return {"graph": PEMSDataset.to_tensor(self.graph), "flow_x": data_x, "flow_y": data_y}
    @staticmethod
    def slice_data(data, history_length, index, train_mode):
        """
        :param data: np.array, normalized traffic data.
        :param history_length: int, length of history data to be used.
        :param index: int, index on temporal axis.
        :param train_mode: str, ["train", "test"].
        :return:
            data_x: np.array, [N, H, D].
            data_y: np.array [N, D].
        """
        if train_mode == "train":
            start_index = index
            end_index = index + history_length
        elif train_mode == "test":
            start_index = index - history_length
            end_index = index
        else:
            raise ValueError("train model {} is not defined".format(train_mode))
        data_x = data[:, start_index: end_index]
        data_y = data[:, end_index]
        return data_x, data_y
    @staticmethod
    def pre_process_data(data, norm_dim):
        """
        :param data: np.array, original traffic data without normalization.
        :param norm_dim: int, normalization dimension.
        :return:
            norm_base: list, [max_data, min_data], data of normalization base.
            norm_data: np.array, normalized traffic data.
        """
        norm_base = PEMSDataset.normalize_base(data, norm_dim)  # find the normalize base
        norm_data = PEMSDataset.normalize_data(norm_base[0], norm_base[1], data)  # normalize data

        return norm_base, norm_data
    @staticmethod
    def normalize_base(data, norm_dim):
        """
        :param data: np.array, original traffic data without normalization.
        :param norm_dim: int, normalization dimension.
        :return:
            max_data: np.array
            min_data: np.array
        """
        max_data = np.max(data, norm_dim, keepdims=True)  # [N, T, D] , norm_dim=1, [N, 1, D]
        min_data = np.min(data, norm_dim, keepdims=True)
        return max_data, min_data
    @staticmethod
    def normalize_data(max_data, min_data, data):
        """
        :param max_data: np.array, max data.
        :param min_data: np.array, min data.
        :param data: np.array, original traffic data without normalization.
        :return:
            np.array, normalized traffic data.
        """
        mid = min_data
        base = max_data - min_data
        normalized_data = (data - mid) / base

        return normalized_data
    @staticmethod
    def recover_data(max_data, min_data, data):
        """
        :param max_data: np.array, max data.
        :param min_data: np.array, min data.
        :param data: np.array, normalized data.
        :return:
            recovered_data: np.array, recovered data.
        """
        mid = min_data
        base = max_data - min_data
        recovered_data = data * base + mid
        return recovered_data
    @staticmethod
    def to_tensor(data):
        return torch.tensor(data, dtype=torch.float)
def get_loader(ds_name="PEMS04"):
    num_nodes = 1 if ds_name == 'PEMS04' else 1
    train_data = PEMSDataset(data_path=[distance_path, 
                                        data_path_npz_2], num_nodes=num_nodes,
                             divide_days=[25,15],
                             time_interval=2, history_length=6,
                             train_mode="train")

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    test_data = PEMSDataset(data_path=[distance_path, 
                                        data_path_npz_2], num_nodes=num_nodes,
                            divide_days=[25,15],
                            time_interval=2, history_length=6,
                            train_mode="test")
    test_loader = DataLoader(test_data, batch_size=12, shuffle=False)
    return train_loader, test_loader

In [ ]:
seed = 2020
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
plt.rcParams['font.sans-serif'] = ['simhei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# data
train_loader, test_loader = get_loader('PEMS04')
gcn = GCN(6, 3, 1)
chebnet = ChebNet(6, 3, 1, 1)
gat = GAT(6, 3, 1)
mlp=MLP(6,1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = [
    chebnet.to(device),
    gat.to(device),
    gcn.to(device),
    #mlp.to(device)
]
all_predict_values = []
epochs = 50
for i in range(len(models)):
    model = models[i]
    criterion = nn.MSELoss().to(device)
    optimizer = optim.Adam(params=model.parameters(), lr=3e-2)
    model.train()
    for epoch in range(epochs):
        epoch_loss, epoch_mae, epoch_rmse, epoch_mape = 0.0, 0.0, 0.0, 0.0
        num = 0
        start_time = time.time()
        for data in train_loader:  # ["graph": [B, N, N] , "flow_x": [B, N, H, D], "flow_y": [B, N, 1, D]]
            data['graph'], data['flow_x'], data['flow_y'] = data['graph'].to(device), data['flow_x'].to(device), data['flow_y'].to(device)
            predict_value = model(data)  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_mae += MAE(data["flow_y"].cpu(), predict_value.cpu())
            epoch_rmse += RMSE(data["flow_y"].cpu(), predict_value.cpu())
            epoch_mape += MAPE(data["flow_y"].cpu(), predict_value.cpu())

            epoch_loss += loss.item()
            num += 1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        end_time = time.time()
        epoch_mae = epoch_mae / num
        epoch_rmse = epoch_rmse / num
        epoch_mape = epoch_mape / num
        print(
            "Epoch: {:04d}, Loss: {:02.4f}, mae: {:02.4f}, rmse: {:02.4f}, mape: {:02.4f}, Time: {:02.2f} mins".format(
                epoch + 1, 10 * epoch_loss / (len(train_loader.dataset) / 64),
                epoch_mae, epoch_rmse, epoch_mape, (end_time - start_time) ))
    
        model.eval()
    with torch.no_grad():
        total_loss = 0.0
        num = 0
        all_predict_value = 0
        all_y_true = 0
        for data in test_loader:
            data['graph'], data['flow_x'], data['flow_y'] = data['graph'].to(device), data['flow_x'].to(device), data['flow_y'].to(device)
            predict_value = model(data)
            if num == 0:
                all_predict_value = predict_value
                all_y_true = data["flow_y"]
            else:
                all_predict_value = torch.cat([all_predict_value, predict_value], dim=0)
                all_y_true = torch.cat([all_y_true, data["flow_y"]], dim=0)
            loss = criterion(predict_value, data["flow_y"])
            total_loss += loss.item()
            num += 1
        epoch_mae = MAE(all_y_true.cpu(), all_predict_value.cpu())
        epoch_rmse = RMSE(all_y_true.cpu(), all_predict_value.cpu())
        epoch_mape = MAPE(all_y_true.cpu(), all_predict_value.cpu())
        print("Test Loss: {:02.4f}, mae: {:02.4f}, rmse: {:02.4f}, mape: {:02.4f}".format(
            10 * total_loss / (len(test_loader.dataset) / 64), epoch_mae, epoch_rmse, epoch_mape))
    all_predict_values.append(all_predict_value.cpu())
    size = sum([param.nelement() for param in model.parameters()])
    print('size=',size)